# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


nltk.download(['punkt', 'stopwords', 'wordnet', 'omw-1.4'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\UsuarioNoAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\UsuarioNoAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\UsuarioNoAdmin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\UsuarioNoAdmin\AppData\Roaming\nltk_data...


True

In [15]:
# load data from database

df = pd.read_sql_table('messages', 'sqlite:///messages.db')
X = df['message'].values
Y = df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']].values

In [16]:
# Split data in train and test
xtrain, xtest, ytrain, ytest=train_test_split(X, Y, train_size=0.80, random_state=0)
print('80% of data for train = ', len(xtrain), ' messages')


80% of data for train =  20972  messages


In [17]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

# Manual clasifier fit

In [18]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [19]:
def tokenize(text):
    # Todo: normalize case and remove punctuation
    text = text.lower()
    text = re.sub(r"[^a-z0-9]", " ", text)
    # tokenize text
    tokens = word_tokenize(text)
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stopwords.words('english')]
    return tokens

In [20]:
# test the tokenize function
print(tokenize(xtrain[0]))

['rue', 'dessalines', 'petit', 'goaves', 'necessary']


In [21]:
# Todo: initialize count vectorizer object and pass the tokenize function to the `tokenizer` parameter
vect = CountVectorizer(tokenizer=tokenize)

In [22]:
# Todo: get counts of each token (word) in text data (corpus) using the fit_transform method
xtrain_V = vect.fit_transform(xtrain)

In [23]:
# convert sparse matrix to numpy array to view the counts of each token (word)
# each row is one line in the text (corpus) and the number is the count of a token
#xtrain_V.toarray()

In [24]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 36)
#pd.set_option('display.width', 1000)
#df = pd.DataFrame(xtrain_V.toarray())
#df

In [25]:
# Todo: initialize tf-idf transformer object. Set smooth_idf parameter to false.
transformer = TfidfTransformer(smooth_idf = False)

In [26]:
# Todo: use counts from count vectorizer results to compute tf-idf values using the fit_transform method
xtrain_TD = transformer.fit_transform(xtrain_V)    

In [27]:
# convert sparse matrix to numpy array to view
# you can see that the counts are normalized
#xtrain_TD.toarray()

In [28]:
#from sklearn.linear_model import LogisticRegression
#lr = LogisticRegression()
#clf = MultiOutputClassifier(estimator=lr)


#from sklearn.svm import SVC
#svc = SVC(gamma="scale")
#clf = MultiOutputClassifier(estimator=svc)

from sklearn.ensemble import RandomForestClassifier
#clf = RandomForestClassifier()

rf = RandomForestClassifier()
clf = MultiOutputClassifier(estimator=rf)


In [29]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns',50)
df = pd.DataFrame(Y)
df.shape
#df

(26216, 36)

In [30]:
xtrain_TD.shape


(20972, 28165)

In [31]:
ytrain.shape

(20972, 36)

In [32]:
clf.fit(xtrain_TD, ytrain)

MultiOutputClassifier(estimator=RandomForestClassifier())

In [33]:
xtest_V = vect.transform(xtest)

In [34]:
xtest_TD = transformer.transform(xtest_V)  

In [35]:
ypred = clf.predict(xtest_TD)

In [36]:
from sklearn.metrics import confusion_matrix

def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    #print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

In [37]:
# display results
accuracy = clf.score(xtest_TD, ytest)
print (accuracy)

0.2801296720061022


### 2. Write a tokenization function to process your text data

In [185]:
def tokenize_old(text):
    '''
    Fuction: tokenize the message
    Input: text to tokenize
    Output: list of tokens
    '''
    lemmatizer = WordNetLemmatizer()
    #text = X[0]
    text = text.lower() # Convert all letert to low.
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)  # Anything that isn't A through Z or 0 through 9 will be replaced by a space
    tokens = word_tokenize(text)
    tokens_wo_stopwords = [token for token in tokens if token not in stopwords.words("english")]
    #stemmed = [PorterStemmer().stem(w) for w in tokens_wo_stopwords]
    stemmed = [lemmatizer.lemmatize(w) for w in tokens_wo_stopwords]
    return stemmed

In [186]:
def tokenize(text):
    text = text.lower() # Convert all leters to low case.
    text = re.sub(r"[^a-z0-9]", " ", text) # Anything that isn't a through z or 0 through 9 will be replaced by a space
    tokens = word_tokenize(text) # tokenize text
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stopwords.words('english')] # lemmatize and remove stop words
    return tokens

In [187]:
X[3]

'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.'

In [188]:
tokenize(X[3])

['un',
 'report',
 'leogane',
 '80',
 '90',
 'destroyed',
 'hospital',
 'st',
 'croix',
 'functioning',
 'need',
 'supply',
 'desperately']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [75]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier

# Instantiate transformers and classifier
vect = CountVectorizer(tokenizer=tokenize) #Bag of Words

train_vector = vect.fit_transform(xtrain) # Here we generate a vector for each message, based on all the messages in X_train.

NameError: name 'xtrain' is not defined

In [ ]:
#import pandas as pd
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)
df = pd.DataFrame(train_vector.toarray())
df               
#train_vector.toarray()

In [ ]:
tfidf = TfidfTransformer(smooth_idf = False) #TF-IDF values
#from sklearn.svm import SVC
#svc = SVC(gamma="scale")
#clf = MultiOutputClassifier(estimator=svc)

# Split data in train and test
xtrain, xtest, ytrain, ytest=train_test_split(X, Y, train_size=0.1, random_state=0)
print('95% of data for train = ', len(xtrain), ' messages')

train_vector = vect.fit_transform(xtrain) # Here we generate a vector for each message, based on all the messages in X_train.
train_tfidf = tfidf.fit_transform(train_vector) # In order to relativize the importance of words that are more common than others, we apply the tfidf method to the message vectors.

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
clf = MultiOutputClassifier(estimator=lr)

In [ ]:
print(clf)

In [ ]:
train_tfidf.shape

In [ ]:
train_tfidf.toarray()

In [ ]:
ytrain.shape

In [ ]:
ytrain.head()

In [ ]:
clf.fit(train_tfidf, ytrain)

In [ ]:
# Fit or train the classifier
from sklearn.datasets import make_multilabel_classification
x, y = make_multilabel_classification(n_samples=5000, n_features=8,

                                      n_classes=36, random_state=0)

clf.fit(x, y)

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
pipeline = 






### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.